In [1]:
from datetime import datetime
from bs4 import BeautifulSoup
import requests
import time, os
import pickle
import pandas as pd
import glob
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from sqlalchemy import create_engine

In [ ]:
chromedriver = "C:\Program Files (x86)\Google\Chrome\Application\chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [ ]:
def download_urls(main_url, chromedriver):
    ''' Function that takes in the County main sold homes page, goes through other county pages and download all csv data'''
    time.sleep(2)
    driver = webdriver.Chrome(chromedriver)
    driver.get(main_url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    download = "https://www.redfin.com" + soup.find(class_='DownloadAndSave').find('a')['href']
    urls= soup.find(class_="PagingControls").find_all('a')
    for index, i in enumerate(urls):
        urls[index] = i['href']
    driver.close()
    time.sleep(1)
    driver = webdriver.Chrome(chromedriver)
    driver.get(download)
    time.sleep(2)
    driver.close()
    for i in urls[1:]:
        driver = webdriver.Chrome(chromedriver)
        url = "https://www.redfin.com" + i
        driver.get(url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        download = "https://www.redfin.com" + soup.find(class_='DownloadAndSave').find('a')['href']
        driver.close()
        time.sleep(1)
        driver = webdriver.Chrome(chromedriver)
        driver.get(download)
        time.sleep(2)
        driver.close()  

In [ ]:
def get_house_info(url, chromedriver):
    ''' Function that takes in the url of a sold home page and returns all house info in a dictionary'''
    time.sleep(1)
    driver = webdriver.Chrome(chromedriver)
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    house_facts = {}
    
    if len(soup.find_all(class_="row PropertyHistoryEventRow")) != 0:
        house_facts['Year Built'] = soup.find(class_="facts-table").find(text="Year Built").next.text
        if house_facts['Year Built'] == "—":
            house_facts['Year Built'] = soup.find(text="Year Built").next.text
        house_facts['Style'] = soup.find(class_="facts-table").find(text="Style").next.text
        house_facts['Year Renovated'] = soup.find(class_="facts-table").find(text="Year Renovated").next.text
        house_facts['County'] = soup.find(class_="facts-table").find(text="County").next.text


        if soup.find(class_="sold-row row PropertyHistoryEventRow") is None:
            house_facts['sold_date'] = soup.find(class_= "home-sash-container small").next.text[5:]
        else:
            house_facts['sold_date'] = soup.find(class_="sold-row row PropertyHistoryEventRow").find(class_="col-4").text[:-4]
        if soup.find(class_="sold-row row PropertyHistoryEventRow") is None: 
            house_facts['sold_price'] = soup.find(class_="statsValue").text
        else:
            house_facts['sold_price'] = soup.find(class_="sold-row row PropertyHistoryEventRow").find(class_="price-col number").text.strip()
        for i in soup.find_all(class_="row PropertyHistoryEventRow"):
            if "Listed" in i.find(class_="description-col col-4").find('div').text:
                house_facts['listed_price'] = i.find(class_="price-col number").text
                house_facts['listed_date'] = i.find(class_="col-4").text[:-4]
                break 

        try:
            house_facts['Redfin_estimate'] = soup.find(text = "Redfin Estimate").parent.parent.parent.parent.find(class_="content text-right").text
        except:
            house_facts['Redfin_estimate'] = "NA"
    
        driver.close()
        
        return house_facts
    else:
        return 0
    

In [ ]:
#Main pages for counties
counties = ["https://www.redfin.com/county/118/WA/King-County/recently-sold",
            "https://www.redfin.com/county/321/CA/Los-Angeles-County/recently-sold",
            "https://www.redfin.com/county/727/IL/Cook-County/recently-sold",
            "https://www.redfin.com/county/2740/TX/Harris-County/recently-sold",
            "https://www.redfin.com/county/220/AZ/Maricopa-County/recently-sold",
            "https://www.redfin.com/city/16904/CA/San-Diego/recently-sold",
            "https://www.redfin.com/county/332/CA/Orange-County/recently-sold",
            "https://www.redfin.com/county/479/FL/Miami-Dade-County/recently-sold",
            "https://www.redfin.com/city/30794/TX/Dallas/recently-sold",
            "https://www.redfin.com/county/1968/NY/Kings-County/recently-sold",
            "https://www.redfin.com/city/15935/CA/Riverside/recently-sold",
            "https://www.redfin.com/neighborhood/43309/NY/New-York/Queens/recently-sold",
            "https://www.redfin.com/city/16659/CA/San-Bernardino/recently-sold",
            "https://www.redfin.com/county/2859/TX/Tarrant-County/recently-sold",
            "https://www.redfin.com/county/2654/TX/Bexar-County/recently-sold",
            "https://www.redfin.com/county/442/FL/Broward-County/recently-sold",
            "https://www.redfin.com/city/17675/CA/Santa-Clara/recently-sold",
            "https://www.redfin.com/county/1429/MI/Wayne-County/recently-sold",
            "https://www.redfin.com/county/303/CA/Alameda-County/recently-sold",
            "https://www.redfin.com/city/30749/NY/New-York/recently-sold",
            "https://www.redfin.com/county/1342/MA/Middlesex-County/recently-sold",
            "https://www.redfin.com/county/2411/PA/Philadelphia-County/recently-sold",
            "https://www.redfin.com/county/336/CA/Sacramento-County/recently-sold",
            "https://www.redfin.com/county/486/FL/Palm-Beach-County/recently-sold"]

In [ ]:
#download counties files
for county in counties:
    download_urls(county, chromedriver)

In [2]:
#Aggregate all downloaded data and load into pandas dataframe
files = []
for file in glob.glob("/Users/elmog/Downloads/redfin*"):
    files.append(pd.read_csv(file))
df = pd.concat(files)  

In [3]:
#change column names
df.columns = ['sale_type', 'sold_date', 'property_type', 'address', 'city',
       'state', 'zip_code', 'sold_price', 'beds', 'baths',
       'location', 'sqft', 'lot_size', 'year_built', 'days_on_market',
       'price_sqft', 'HOA', 'status', 'open_house_start',
       'open_house_end','url','source', 'MLS', 'favorite', 'interested', 'latitude', 'longitude']

In [ ]:
# #get additional homes info (work in progress, not included in current version)
# houses_urls = df.url.tolist()
# house_info = []
# for house in houses_urls:
#     house_info.append(get_house_info(house, chromedriver))
# #merge two dataframes to add new info

In [4]:
#convert sold_date to datetime
df['sold_date'] = df['sold_date'].astype(str)

def convert_date(row):
    try:
        l = row['sold_date'].split('-')
        l[0] = l[0][:3]
        l = " ".join(l)
        l = datetime.strptime(l, '%b %d %Y').date()
    except:
        l = row['sold_date']
    return l
df['sold_date'] = df.apply(convert_date, axis=1)

In [9]:
#save into sql db
engine = create_engine('sqlite:///sold_houses.db', echo=True)
sqlite_connection =engine.connect()
sqlite_table = "houses"
df.to_sql(sqlite_table, sqlite_connection, if_exists = 'append')

2022-04-20 01:22:02,733 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("houses")
2022-04-20 01:22:02,736 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-20 01:22:02,740 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("houses")
2022-04-20 01:22:02,741 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-04-20 01:22:02,755 INFO sqlalchemy.engine.Engine 
CREATE TABLE houses (
	"index" BIGINT, 
	sale_type TEXT, 
	sold_date TEXT, 
	property_type TEXT, 
	address TEXT, 
	city TEXT, 
	state TEXT, 
	zip_code TEXT, 
	sold_price FLOAT, 
	beds FLOAT, 
	baths FLOAT, 
	location TEXT, 
	sqft FLOAT, 
	lot_size FLOAT, 
	year_built FLOAT, 
	days_on_market FLOAT, 
	price_sqft FLOAT, 
	"HOA" FLOAT, 
	status TEXT, 
	open_house_start FLOAT, 
	open_house_end FLOAT, 
	url TEXT, 
	source TEXT, 
	"MLS" TEXT, 
	favorite TEXT, 
	interested TEXT, 
	latitude FLOAT, 
	longitude FLOAT
)


2022-04-20 01:22:02,758 INFO sqlalchemy.engine.Engine [no key 0.00326s] ()
2022-04-20 01:22:02,809 INFO sqlalchemy.engine

In [10]:
sqlite_connection.close()